In [1]:
# Imports
from tqdm import tqdm_notebook as tqdm

import sys
sys.path.append("../")

from algorithms.flow import flow
from data_structures.location_graph import LocationGraph
from utils.dummy_writer import DummyWriter
from utils.image_writer import ImageWriter
from utils.report_settings import report_settings
from utils.load_image import load_and_crop_data
from utils.colour import get_colour_function
from utils.publish import make_video

In [2]:
# Params
size = (2560, 1440)
frequency = 5000
job_name = "rect"
report_settings(size, frequency, job_name)
height_map = load_and_crop_data(size, (0, 0)).tolist()

Num images: 737
Total data size: 2.72 GB


In [3]:
%%time
graph = LocationGraph(height_map)

Creating graph



Sorting nodes
Building linked list
CPU times: user 1min 38s, sys: 2.55 s, total: 1min 40s
Wall time: 1min 40s


In [4]:
%%time
nodes_with_flow = flow(graph, DummyWriter(), size)

Running flow simulation



CPU times: user 14.9 s, sys: 68.7 ms, total: 15 s
Wall time: 14.9 s


In [5]:
flows = [n.flow for n in nodes_with_flow.ascending()]
imgWriter = ImageWriter(size, frequency, job_name, get_colour_function(flows))

Max flow: 1558.9398215354938
Max log(flow): 7.3517612676299375
Writer will save images to /home/andrew/personal/river-flow/data/rect/


In [7]:
%%time
for node in tqdm(graph.descending(), total=size[0]*size[1], unit=" nodes"):
    imgWriter.update(node)


CPU times: user 9.23 s, sys: 12.5 s, total: 21.7 s
Wall time: 21.8 s


In [ ]:
%%time
make_video(job_name)